Загружаем исходные данные в colab.

In [1]:
from google.colab import files
uploaded = files.upload()

Saving dataset(1).csv to dataset(1).csv


Загружаем данные в датафрейм

In [ ]:
import io
import pandas as pd
import numpy as np
import plotly.express as px

df = pd.read_csv(io.BytesIO(uploaded['dataset (3).csv']))

df

NameError: name 'uploaded' is not defined

In [ ]:
df['event_date'] = pd.to_datetime(df['event_date'])
df['install_date'] = pd.to_datetime(df['install_date'])
df.dtypes

Сегодня мы посмотрим метрики проекта в разрезе версионности - сравним их для различных билдов.

Для начала посмотрим наименования версий проекта.

In [ ]:
df['app_version'].unique()

Заменим nan значения на unknown

In [ ]:
df['app_version'] = df['app_version'].fillna('unknown')
df['app_version'].unique()

Для того, чтобы получить чистые данные при расчете метрик привяжем к каждому пользователю одну версию - ту, с которой он зашел в игру (в процессе пользования продуктом пользователи обновляют приложение и соответственно у каждого из них может быть несколько версий за лайфтайм)

In [ ]:
df_start_ver = df[df['event_date'] == df['install_date']][['appsflyer_id', 'app_version']].rename(columns = {'app_version':'start_version'})
df = pd.merge(df, df_start_ver, on='appsflyer_id')
df

Так как мы будем рассчитывать в том числе retention rate и LTV по каждой из версий, нам необходимо ограничить датасет временными рамками по когортам срок жизни которых достиг 30 дней и более.
Напомним, что максимальный event date = 2022-07-31

In [ ]:
df_versions = df[(df['install_date'] >= '2022-04-01') & (df['install_date'] <= '2022-07-01')][['event_date', 'start_version', 'install_date', 'appsflyer_id', 'is_organic', 'revenue_usd', 'ad_revenue_usd', 'sessions', 'playtime']]
df_versions['activity_day'] = (df_versions['event_date']-df_versions['install_date']).dt.days
df_versions

Посмотрим инсталлы по каждой версии и оставим для расчета топ-3 по инсталлам

In [ ]:
df_top = df_versions.groupby('start_version', as_index=False).agg({'appsflyer_id':'nunique'}).rename(columns = {'appsflyer_id':'users'}).sort_values(by = 'users', ascending=False)
top = list(df_top.head(3)['start_version'].values)
top

In [ ]:
df_versions = df_versions[df_versions['start_version'].isin(top)]
df_versions

Рассчитаем retention rate для каждого билда за 0-30 день активности

In [ ]:
df_ret = df_versions[(df_versions['activity_day']>=0) & (df_versions['activity_day']<=30)].groupby(['start_version', 'activity_day'], as_index=False).agg({'appsflyer_id':'nunique'}).rename(columns = {'appsflyer_id':'users'})
df_ret

In [ ]:
df_installs = df_ret.groupby('start_version', as_index=False).agg({'users':'max'}).rename(columns = {'users':'installs'})

df_ret = pd.merge(df_ret, df_installs, on='start_version')
df_ret['ret_rate'] = df_ret['users'] / df_ret['installs']

df_ret

In [ ]:
fig1 = px.line(df_ret, x='activity_day', y='ret_rate', color='start_version')
fig1.show()

Какие выводы можно сделать из полученных результатов?

Для лучшего понимания деталей посмотрим на retention rate по версиям для 1-10 дней активности

In [ ]:
fig2 = px.line(df_ret, x='activity_day', y='ret_rate', color='start_version', range_x=[1,10], range_y=[0,0.3])
fig2.show()

Посмотрим средние значения метрик вовлеченности и LTV по топ-3 версиям

In [ ]:
df_eng = df_versions.groupby(['appsflyer_id', 'start_version'], as_index=False).agg({'sessions':'sum', 'playtime':'sum', 'revenue_usd':'sum', 'ad_revenue_usd':'sum'})
df_eng = df_eng.groupby('start_version', as_index = False).agg({'appsflyer_id':'nunique', 'sessions':'sum', 'playtime':'sum', 'revenue_usd':'sum', 'ad_revenue_usd':'sum'}).rename(columns = {'appsflyer_id':'users'})

df_eng

In [ ]:
df_eng['sessions_per_user'] = df_eng['sessions'] / df_eng['users']
df_eng['playtime_per_user'] = df_eng['playtime'] / df_eng['users']
df_eng['average_session_length'] = df_eng['playtime'] / df_eng['sessions']
df_eng['ltv'] = (df_eng['revenue_usd']+df_eng['ad_revenue_usd']) / df_eng['users']

df_eng

Сделайте выводы по полученным результатам.

# Домашнее задание

*Просчитайте и проанализируйте основные метрики для топ-5 билдов по инсталлам.*

Сформируем список топ-5 билдов по инсталлам

In [ ]:
df_installs = df_ret.groupby('start_version', as_index=False).agg({'users':'max'}).rename(columns = {'users':'installs'})

df_ret = pd.merge(df_ret, df_installs, on='start_version')
df_ret['ret_rate'] = df_ret['users'] / df_ret['installs']

df_ret

"payments", "revenue_usd", "ad_revenue_usd", "sessions", "playtime"

In [ ]:
df_top_inst = df.groupby('app_version', as_index=False).agg({'appsflyer_id':'nunique'}).rename(columns = {'appsflyer_id':'users'}).sort_values(by='users', ascending=False).head(5)
top_inst